In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
%matplotlib inline

module_path = os.path.abspath(os.path.join("../lib"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from shrinkage_estimators import covariance_linear_shrinkage
from shrinkage_estimators import precision_shrinkage
from shrinkage_estimators import NERCOME

In [ ]:
data_path = "../output/BOSS_DR12_NGC_z1/"
p = 18
T = np.loadtxt(os.path.join(data_path, f"T_{p}_{p}.matrix"))

In [ ]:
def estimate_matrices(n):
    save_path = os.path.join(data_path, f"matrices/n{n}/")
    mocks_path = os.path.join(data_path, f"mocks/n{n}/")
    r = len([f for f in os.listdir(mocks_path) if not f.startswith('.')])
    
    for v in range(r):
        print(f"Run {v+1}")
        P = np.loadtxt(os.path.join(mocks_path, f"P_{p}_{n}_v{v+1}.matrix"))
    
        # Sample cov estimate
        S = np.cov(P)
        np.savetxt(os.path.join(save_path, "cov_sample", f"cov_{p}_{p}_sample_{n}_v{v+1}.matrix"), S)
        
        # Shrinkage cov estimates, empirical target = diag(S)
        C_emp, lmbda_emp = covariance_linear_shrinkage.linear_shrinkage(P)
        np.savetxt(os.path.join(save_path, "cov_shrinkage_emp", f"cov_{p}_{p}_shrinkage_emp_{n}_v{v+1}.matrix"), 
                   C_emp,
                   header=f"lambda={lmbda_emp}")
        
        # Shrinkage cov estimates, analytical target
        C_ana, lmbda_ana = covariance_linear_shrinkage.linear_shrinkage(P, target=T)
        np.savetxt(os.path.join(save_path, "cov_shrinkage_ana", f"cov_{p}_{p}_shrinkage_ana_{n}_v{v+1}.matrix"),
                   C_ana,
                   header=f"lambda={lmbda_ana}")
    
        # Shrinkage precision matrix estimates, empirical target = evals(S^{-1})
        target = np.diag(np.linalg.eigvalsh(np.linalg.inv(S)))
        psi_emp, alpha_emp, beta_emp = precision_shrinkage.shrinkage(P, target)
        np.savetxt(os.path.join(save_path, "pre_shrinkage_emp", f"pre_{p}_{p}_shrinkage_emp_{n}_v{v+1}.matrix"),
                   psi_emp,
                   header=f"alpha={alpha_emp}, beta={beta_emp}")
        
        # Shrinkage precision matrix estimates, analytical target
        psi_ana, alpha_ana, beta_ana = precision_shrinkage.shrinkage(P, np.linalg.inv(T))
        np.savetxt(os.path.join(save_path, "pre_shrinkage_ana", f"pre_{p}_{p}_shrinkage_ana_{n}_v{v+1}.matrix"),
                   psi_ana,
                   header=f"alpha={alpha_ana}, beta={beta_ana}")
        
        
        # NERCOME cov estimate
        C_nercome, s_min = NERCOME.NERCOME(P, all_splits=(n <= 50)) # Only do all splits if n at most 50
        np.savetxt(os.path.join(save_path, "cov_NERCOME", f"cov_{p}_{p}_NERCOME_{n}_v{v+1}.matrix"),
                   C_nercome,
                   header=f"s_min={s_min}")

In [ ]:
estimate_matrices(24)
estimate_matrices(30)
estimate_matrices(2048)